In [2]:
# Directory to search
#directory_to_search = "/home/elias/IdeaProjects/systemds/scripts/builtin"
directory_to_search = "/Users/eliasstrauss/Desktop/TU/systemds-fork/scripts/builtin"

matches = []

# Regex pattern for matching function and return statements
pattern = re.compile(r"(?P<func>\b\w+\b\s*=\s*function\s*\([^)]*\)\s*)"  # Match 'fname = function (...)'
                     r"(?:(?P<return>return\s*\([^)]*\)))?",             # Match 'return (...)', optional
                     re.DOTALL)                                          # Enable dotall to match across lines

# Walk through the directory
for root, _, files in os.walk(directory_to_search):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
                # replace matrix(nrow(X), ncol(X)) by matrix to avoid the pattern function (...) to pick up the the first closing bracket from nrow(X)
                # this quick hack is easier than counting opening and closing brackets...
                pattern_nrow = r"nrow\([^)]*\)"
                content = re.sub(pattern_nrow, "nrow", content)
                pattern_ncol = r"ncol\([^)]*\)"
                content = re.sub(pattern_ncol, "ncol", content)
                pattern_matrix = r"matrix\([^)]*\)"
                content = re.sub(pattern_matrix, "matrix", content)
                pattern_frame = r"as.frame\([^)]*\)"
                content = re.sub(pattern_frame, "frame", content)
                pattern_list = r"list\(\)"
                content = re.sub(pattern_list, "list", content)

                # Find all matches in the file
                no_match = True
                for match in pattern.finditer(content):
                    function_header = match.group()
                    matches.append({
                        "file_name": file_path,
                        "function_header": function_header
                    })
                    no_match = False
                if no_match:
                    print(file_path)
        except (UnicodeDecodeError, IOError):
            print(f"Could not read file: {file_path}")
            
results = matches
print(len(results), len([1 for _ in files for _, _, files in os.walk(directory_to_search)]))

434 192


In [3]:
results_inner = [r for r in results if r["function_header"][1] != "_"]
results  = [r for r in results if r["function_header"][1] == "_"]
len(results)

203

In [4]:
# check that there were no errors while parsing
for result in results:
    header = result["function_header"]
    if header.count("(") != header.count(")"):
        print(header)

In [5]:
# Print results
for result in results:
    print(f"File: {result['file_name']}")
    print(f"Function Header: {result['function_header']}")
    print("-" * 40)

File: /Users/eliasstrauss/Desktop/TU/systemds-fork/scripts/builtin/mse.dml
Function Header: m_mse = function(Matrix[Double] X, Matrix[Double] Y, Matrix[Double] P = matrix) 
  return (Matrix[Double] Z, Matrix[Double] Q)
----------------------------------------
File: /Users/eliasstrauss/Desktop/TU/systemds-fork/scripts/builtin/executePipeline.dml
Function Header: f_executePipeline = function(Frame[String] pipeline, Matrix[Double] X,  Matrix[Double] Y,
  Matrix[Double] Xtest,  Matrix[Double] Ytest, List[Unknown] metaList, Matrix[Double] hyperParameters, Matrix[Double] hpForPruning = as.matrix,
  Matrix[Double] changesByOp = as.matrix, Integer flagsCount, Boolean test = FALSE, Boolean verbose)
  return (Matrix[Double] Xtrain, Matrix[Double] Ytrain, Matrix[Double] Xtest, Matrix[Double] Ytest,
    Double t2, Matrix[Double] hpForPruning, Matrix[Double] changesByOp, Double changesAll, List[Unknown] internalStates)
----------------------------------------
File: /Users/eliasstrauss/Desktop/TU/sy

In [6]:
unique_parameters = dict()
intput_parameter_filter = re.compile(r"function\s*\(([^)]*)\)")
param_with_default_pattern = re.compile(r"(\w+)\s+(\w+)\s*=\s*(.+)")
for result in results:
    function_header = result["function_header"]

    # Search for the parameters inside "function(...)"
    param_match = intput_parameter_filter.search(function_header)
    if param_match:
        param_str = param_match.group(1)  # The content inside the parentheses

        # Split the parameters by commas and process each one
        params = [p.strip() for p in param_str.split(",") if p.strip()]
        for param in params:
            if not '=' in param:
                parts = param.split()
                if not parts[-1] in unique_parameters:
                    unique_parameters[parts[-1]] = 1
                else:
                    unique_parameters[parts[-1]] += 1
            else:
               # Extract the parameter name from possible formats:
                # NAME, DATATYPE NAME, DATATYPE NAME=DEFAULT, DATATYPE NAME = DEFAULT
                param = param.replace('[', '').replace(']','')
                match = param_with_default_pattern.match(param)
                if match:
                    param_name = match.group(2)
                    if not param_name in unique_parameters:
                        #print(param_name, "[{}]".format(param))
                        unique_parameters[param_name] = 1
                    else:
                        unique_parameters[param_name] += 1
                else:
                    print("Warning no match for: " + param)
print(len(unique_parameters))

340


In [7]:
names = list(unique_parameters.keys())
names.sort()

In [8]:
for name in names:
    print("{name: <28} : {c}".format(name=name, c=unique_parameters[name]))

A                            : 3
B                            : 4
C                            : 4
CL                           : 1
Centering                    : 3
Clusters                     : 2
D                            : 4
F                            : 2
F1                           : 1
G                            : 3
GI                           : 1
Graph                        : 1
H                            : 2
I                            : 2
IQR                          : 1
K                            : 3
L                            : 2
LHSfeatures                  : 1
LHSthreshold                 : 1
M                            : 10
Mask                         : 2
Mu                           : 1
Order                        : 1
P                            : 16
Precisions                   : 1
Q                            : 2
Q1                           : 1
Q3                           : 1
QL                           : 2
QU                           : 2
R       

In [9]:
para_by_count = sorted([pair for pair in unique_parameters.items()], key=lambda pair: (pair[1],pair[0]), reverse=False)

In [10]:
for n, c in para_by_count:
    print("{name: <28} : {c}".format(name=n, c=c))

CL                           : 1
F1                           : 1
GI                           : 1
Graph                        : 1
IQR                          : 1
LHSfeatures                  : 1
LHSthreshold                 : 1
Mu                           : 1
Order                        : 1
Precisions                   : 1
Q1                           : 1
Q3                           : 1
RHSfeatures                  : 1
RHSthreshold                 : 1
S1                           : 1
S2                           : 1
SI                           : 1
Scid                         : 1
T                            : 1
T1                           : 1
T2                           : 1
Test                         : 1
Train                        : 1
W1                           : 1
W2                           : 1
W3                           : 1
W4                           : 1
Xcid                         : 1
Ycid                         : 1
addedE                       : 1
addedX    

In [11]:
names_prev = names

In [12]:
not_in_new = [n for n in names_prev if n not in names]

In [13]:
len(not_in_new)

0

In [14]:
not_in_prev = [n for n in names if n not in names_prev]

In [15]:
len(not_in_prev)

0

In [16]:
138 - 76

62

In [17]:
for name in not_in_prev:
    print(name)

In [18]:
for name in not_in_new:
    print(name)